# L-layer network. 

* We will work on classifying cat pictures with L layers of neural network 
* First, we will write some function that will help make the complete model 
* Broadly, we will follow the steps below: 
    - Initialize parameters for all layers 
    - Forward propagation for Linear -> ReLu layers, i.e. from layer 1 to laye L-1
    - Forward propagation for Linear -> Sigmoid layers, i.e. for layer L.
    - Compute cost function(cross entropy loss)
    - Backward propagation for Linear -> Sigmoid layer L.
    - Backward propagation for Linear -> ReLu layers, i.e. from L-1 to 1.
    - we update parameter at each step during this process 

 * We will be using following function for this work 
     - Initilaize_parameters 
     - L_model_forward 
     - Compute_cost 
     - L_model_backward 
     - Update_parameters 
     - sigmoid 
     - Relu 
     - Sigmoid_backward 
     - ReLu_backward

In [1]:
# imports 
import numpy as np
import matplotlib.pyplot as plt
import h5py

In [2]:
# Activation functions for different layers 

## Relu and its back prop implementation 

def relu(Z):
    """
    Implement ReLu activation fucntion 
    
    Argument: 
    Z : array of matrix 
    
    Return : 
    A : a rectified linear unit output. 
    Activation_cache : cached value of Z in a dictionary, that will be used during back propagation 
    """
    A = np.maximum(0, Z) 
    
    assert(A.shape == Z.shape)
    
    cache = Z
    return A, cache

def relu_backward(dA, cache):
    """
    Implement relu backward for single unit 
    
    Arguments: 
    dA : 
    cache : 
    
    Returns: 
    dZ : drivative of the relu function 
    """
    
    Z = cache 
    dZ = np.array(dA, copy=True)
    
    # for relu, we only need to take care of the less than zero terms and set them to zero 
    dZ[Z<=0] = 0
    assert(dZ.shape == Z.shape)
    
    return dZ

# sigmoid and its backprop implementation

def sigmoid(Z): 
    """
    Implement sigmoid function on array. 
    
    Argument: 
    Z : weighted input array before activation 
    
    Returns: 
    A : activation value A after application of sigmoid. 
    cache : value of Z, cached for effective calculation during back propagation
    """
    
    A = 1/(1+np.exp(-Z))
    cache = Z
    
    return A, cache

def sigmoid_backward(dA, cache): 
    """
    Implement derivative of sigmoid. 
    
    Arguments: 
    A : post active gradient of any shape 
    Cache : value Z cached during sigmoid calculation
    
    Returns : 
    Z : derivative of sigmoid, here it will return derivative of Cost.
    """
    
    Z = cache 
    sigmoid = 1/ (1 + np.exp(-Z))
    dZ = dA*sigmoid*(1-sigmoid)
    
    assert(dZ.shape == Z.shape)
    
    return dZ



In [4]:
    
# a function to load data, 
# credit: deeplearnin.ai module 1 week 4 assignments 

def load_data():
    train_dataset = h5py.File('train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

array([[[[ 17,  31,  56],
         [ 22,  33,  59],
         [ 25,  35,  62],
         ...,
         [  1,  28,  57],
         [  1,  26,  56],
         [  1,  22,  51]],

        [[ 25,  36,  62],
         [ 28,  38,  64],
         [ 30,  40,  67],
         ...,
         [  1,  27,  56],
         [  1,  25,  55],
         [  2,  21,  51]],

        [[ 32,  40,  67],
         [ 34,  42,  69],
         [ 35,  42,  70],
         ...,
         [  1,  25,  55],
         [  0,  24,  54],
         [  1,  21,  51]],

        ...,

        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],

        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],

        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
        